# 🎓 CSC582: Relational to Key-Value Database Migration
## E-Commerce Database → Redis Cluster

**Course:** CSC582 - Data Warehouse and Mining Systems

**Mapping Format:** `TableName:TupleID:Attribute` → Value

---

## 📦 Step 1: Install Redis Server

In [ ]:
# Install Redis Server and Python client
!apt-get update -qq
!apt-get install -y redis-server redis-tools > /dev/null
!pip install redis -q

print("✅ Redis installed successfully!")
!redis-server --version

## 🚀 Step 2: Start Redis Server

In [ ]:
import subprocess
import time

# Start Redis server in background
subprocess.Popen(['redis-server', '--daemonize', 'yes'])
time.sleep(2)

# Verify Redis is running
result = subprocess.run(['redis-cli', 'ping'], capture_output=True, text=True)
if result.stdout.strip() == 'PONG':
    print("✅ Redis server is running!")
else:
    print("❌ Redis failed to start")

---
# 📊 Part 1: Relational Database Schema

E-Commerce database with 3 tables:
- **Customer**: Customer information (5 records)
- **Product**: Product catalog (6 records)
- **Order**: Customer orders (7 records)

In [ ]:
import sqlite3
import pandas as pd

# Create SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create tables
cursor.executescript("""
    CREATE TABLE Customer (
        customer_id     INTEGER PRIMARY KEY,
        first_name      TEXT NOT NULL,
        last_name       TEXT NOT NULL,
        email           TEXT UNIQUE NOT NULL,
        phone           TEXT,
        city            TEXT,
        country         TEXT
    );
    
    CREATE TABLE Product (
        product_id      INTEGER PRIMARY KEY,
        product_name    TEXT NOT NULL,
        category        TEXT,
        price           REAL NOT NULL,
        stock_quantity  INTEGER DEFAULT 0
    );
    
    CREATE TABLE Order_ (
        order_id        INTEGER PRIMARY KEY,
        customer_id     INTEGER NOT NULL,
        order_date      TEXT NOT NULL,
        status          TEXT DEFAULT 'pending',
        total_amount    REAL,
        FOREIGN KEY (customer_id) REFERENCES Customer(customer_id)
    );
""")

print("✅ Relational schema created!")
print("\n📋 Tables: Customer, Product, Order")

In [ ]:
# Insert Customer data
customers_data = [
    (1, 'Ahmed', 'Al-Rashid', 'ahmed.rashid@email.com', '+966501234567', 'Riyadh', 'Saudi Arabia'),
    (2, 'Fatima', 'Hassan', 'fatima.hassan@email.com', '+966502345678', 'Jeddah', 'Saudi Arabia'),
    (3, 'Mohammed', 'Al-Saud', 'mohammed.saud@email.com', '+966503456789', 'Dammam', 'Saudi Arabia'),
    (4, 'Sara', 'Abdullah', 'sara.abdullah@email.com', '+966504567890', 'Riyadh', 'Saudi Arabia'),
    (5, 'Khalid', 'Omar', 'khalid.omar@email.com', '+966505678901', 'Mecca', 'Saudi Arabia'),
]
cursor.executemany("INSERT INTO Customer VALUES (?,?,?,?,?,?,?)", customers_data)

# Insert Product data
products_data = [
    (101, 'Laptop Pro 15', 'Electronics', 4500.00, 25),
    (102, 'Wireless Mouse', 'Electronics', 150.00, 100),
    (103, 'Office Chair', 'Furniture', 850.00, 30),
    (104, 'Standing Desk', 'Furniture', 2200.00, 15),
    (105, 'Noise-Canceling Headphones', 'Electronics', 1200.00, 50),
    (106, 'USB-C Hub', 'Electronics', 280.00, 75),
]
cursor.executemany("INSERT INTO Product VALUES (?,?,?,?,?)", products_data)

# Insert Order data
orders_data = [
    (1001, 1, '2024-06-01', 'delivered', 4650.00),
    (1002, 2, '2024-06-05', 'delivered', 3050.00),
    (1003, 1, '2024-06-10', 'shipped', 1200.00),
    (1004, 3, '2024-06-15', 'processing', 2480.00),
    (1005, 4, '2024-06-20', 'pending', 850.00),
    (1006, 5, '2024-06-25', 'delivered', 4780.00),
    (1007, 2, '2024-07-01', 'shipped', 430.00),
]
cursor.executemany("INSERT INTO Order_ VALUES (?,?,?,?,?)", orders_data)
conn.commit()

print("✅ Sample data inserted!")
print("   - 5 Customers")
print("   - 6 Products")
print("   - 7 Orders")

In [ ]:
# Display Customer Table
print("👥 CUSTOMER TABLE")
print("="*80)
df_customers = pd.read_sql_query("SELECT * FROM Customer", conn)
display(df_customers)

In [ ]:
# Display Product Table
print("📦 PRODUCT TABLE")
print("="*80)
df_products = pd.read_sql_query("SELECT * FROM Product", conn)
display(df_products)

In [ ]:
# Display Order Table
print("🛒 ORDER TABLE")
print("="*80)
df_orders = pd.read_sql_query("SELECT * FROM Order_", conn)
display(df_orders)

---
# 🔄 Part 2: Mapping Strategy

## Key Format: `TableName:TupleID:Attribute` → Value

Each column in a relational row becomes a **separate key-value pair**.

### Example - Customer Table Mapping:

**Relational Row:**
| customer_id | first_name | last_name | email | phone | city | country |
|-------------|------------|-----------|-------|-------|------|--------|
| 1 | Ahmed | Al-Rashid | ahmed.rashid@email.com | +966501234567 | Riyadh | Saudi Arabia |

**Key-Value Pairs:**
| Key | Value |
|-----|-------|
| Customer:1:first_name | "Ahmed" |
| Customer:1:last_name | "Al-Rashid" |
| Customer:1:email | "ahmed.rashid@email.com" |
| Customer:1:phone | "+966501234567" |
| Customer:1:city | "Riyadh" |
| Customer:1:country | "Saudi Arabia" |

In [ ]:
import redis

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Verify connection
print("🔗 Connected to Redis:", r.ping())

---
# 📤 Part 3: Data Migration to Redis

Using format: **TableName:TupleID:Attribute** → Value

In [ ]:
print("="*70)
print("🚀 MIGRATING DATA TO REDIS")
print("Key Format: TableName:TupleID:Attribute → Value")
print("="*70)

# Migrate Customers
print("\n👥 Migrating CUSTOMER Table...")
print("-"*70)
cursor.execute("SELECT * FROM Customer")
for row in cursor.fetchall():
    cid = row[0]
    
    r.set(f"Customer:{cid}:first_name", row[1])
    print(f"   SET Customer:{cid}:first_name → \"{row[1]}\"")
    
    r.set(f"Customer:{cid}:last_name", row[2])
    print(f"   SET Customer:{cid}:last_name → \"{row[2]}\"")
    
    r.set(f"Customer:{cid}:email", row[3])
    print(f"   SET Customer:{cid}:email → \"{row[3]}\"")
    
    r.set(f"Customer:{cid}:phone", row[4])
    print(f"   SET Customer:{cid}:phone → \"{row[4]}\"")
    
    r.set(f"Customer:{cid}:city", row[5])
    print(f"   SET Customer:{cid}:city → \"{row[5]}\"")
    
    r.set(f"Customer:{cid}:country", row[6])
    print(f"   SET Customer:{cid}:country → \"{row[6]}\"")
    print()

In [ ]:
# Migrate Products
print("📦 Migrating PRODUCT Table...")
print("-"*70)
cursor.execute("SELECT * FROM Product")
for row in cursor.fetchall():
    pid = row[0]
    
    r.set(f"Product:{pid}:product_name", row[1])
    print(f"   SET Product:{pid}:product_name → \"{row[1]}\"")
    
    r.set(f"Product:{pid}:category", row[2])
    print(f"   SET Product:{pid}:category → \"{row[2]}\"")
    
    r.set(f"Product:{pid}:price", str(row[3]))
    print(f"   SET Product:{pid}:price → \"{row[3]}\"")
    
    r.set(f"Product:{pid}:stock_quantity", str(row[4]))
    print(f"   SET Product:{pid}:stock_quantity → \"{row[4]}\"")
    print()

In [ ]:
# Migrate Orders
print("🛒 Migrating ORDER Table...")
print("-"*70)
cursor.execute("SELECT * FROM Order_")
for row in cursor.fetchall():
    oid = row[0]
    
    r.set(f"Order:{oid}:customer_id", str(row[1]))
    print(f"   SET Order:{oid}:customer_id → \"{row[1]}\"")
    
    r.set(f"Order:{oid}:order_date", row[2])
    print(f"   SET Order:{oid}:order_date → \"{row[2]}\"")
    
    r.set(f"Order:{oid}:status", row[3])
    print(f"   SET Order:{oid}:status → \"{row[3]}\"")
    
    r.set(f"Order:{oid}:total_amount", str(row[4]))
    print(f"   SET Order:{oid}:total_amount → \"{row[4]}\"")
    print()

print("="*70)
print("✅ MIGRATION COMPLETE!")
print(f"Total keys in Redis: {r.dbsize()}")
print("="*70)

---
# 🔍 Part 4: Key-Value Operations - GET by Key

In [ ]:
print("📋 GET Customer 1 - All Attributes")
print("="*60)

print(f"GET Customer:1:first_name  → {r.get('Customer:1:first_name')}")
print(f"GET Customer:1:last_name   → {r.get('Customer:1:last_name')}")
print(f"GET Customer:1:email       → {r.get('Customer:1:email')}")
print(f"GET Customer:1:phone       → {r.get('Customer:1:phone')}")
print(f"GET Customer:1:city        → {r.get('Customer:1:city')}")
print(f"GET Customer:1:country     → {r.get('Customer:1:country')}")

In [ ]:
print("📋 GET Product 101 - All Attributes")
print("="*60)

print(f"GET Product:101:product_name    → {r.get('Product:101:product_name')}")
print(f"GET Product:101:category        → {r.get('Product:101:category')}")
print(f"GET Product:101:price           → {r.get('Product:101:price')}")
print(f"GET Product:101:stock_quantity  → {r.get('Product:101:stock_quantity')}")

In [ ]:
print("📋 GET Order 1001 - All Attributes")
print("="*60)

print(f"GET Order:1001:customer_id   → {r.get('Order:1001:customer_id')}")
print(f"GET Order:1001:order_date    → {r.get('Order:1001:order_date')}")
print(f"GET Order:1001:status        → {r.get('Order:1001:status')}")
print(f"GET Order:1001:total_amount  → {r.get('Order:1001:total_amount')}")

---
# 🔎 Part 5: Key-Value Operations - GET by Value (Using KEYS Pattern)

In [ ]:
print("📋 Find all keys for Customer:1")
print("="*60)
print("Command: KEYS Customer:1:*")
print("-"*60)
keys = r.keys("Customer:1:*")
for k in sorted(keys):
    print(f"   {k} → {r.get(k)}")

In [ ]:
print("📋 Find all Customer email addresses")
print("="*60)
print("Command: KEYS Customer:*:email")
print("-"*60)
keys = r.keys("Customer:*:email")
for k in sorted(keys):
    print(f"   {k} → {r.get(k)}")

In [ ]:
print("📋 Find all Product prices")
print("="*60)
print("Command: KEYS Product:*:price")
print("-"*60)
keys = r.keys("Product:*:price")
for k in sorted(keys):
    name_key = k.replace(":price", ":product_name")
    print(f"   {r.get(name_key)}: ${r.get(k)}")

In [ ]:
print("📋 Find all Order statuses")
print("="*60)
print("Command: KEYS Order:*:status")
print("-"*60)
keys = r.keys("Order:*:status")
for k in sorted(keys):
    order_id = k.split(":")[1]
    print(f"   Order {order_id}: {r.get(k)}")

---
# 🔗 Part 6: Simulating JOIN Operation

In [ ]:
print("📋 Simulating JOIN: Get Order 1001 with Customer Details")
print("="*60)

# Step 1: Get customer_id from order
customer_id = r.get("Order:1001:customer_id")
print(f"Step 1: GET Order:1001:customer_id → {customer_id}")

# Step 2: Get customer details using that ID
first_name = r.get(f"Customer:{customer_id}:first_name")
last_name = r.get(f"Customer:{customer_id}:last_name")
print(f"Step 2: GET Customer:{customer_id}:first_name → {first_name}")
print(f"Step 3: GET Customer:{customer_id}:last_name → {last_name}")

# Step 4: Get order details
order_date = r.get("Order:1001:order_date")
total = r.get("Order:1001:total_amount")
status = r.get("Order:1001:status")
print(f"Step 4: GET Order:1001:order_date → {order_date}")
print(f"Step 5: GET Order:1001:total_amount → {total}")
print(f"Step 6: GET Order:1001:status → {status}")

print("\n" + "="*60)
print(f"✅ RESULT: Order 1001")
print(f"   Customer: {first_name} {last_name}")
print(f"   Date: {order_date}")
print(f"   Amount: ${total}")
print(f"   Status: {status}")

---
# 🖥️ Part 7: Redis CLI - Terminal Commands

You can run Redis commands directly using `!redis-cli`

In [ ]:
print("🖥️ REDIS CLI COMMANDS")
print("="*60)

print("\n1. GET Customer:1:first_name")
!redis-cli GET Customer:1:first_name

print("\n2. GET Customer:1:email")
!redis-cli GET Customer:1:email

print("\n3. GET Product:101:product_name")
!redis-cli GET Product:101:product_name

print("\n4. GET Product:101:price")
!redis-cli GET Product:101:price

In [ ]:
print("🖥️ KEYS Command - Find keys by pattern")
print("="*60)

print("\nKEYS Customer:1:*")
!redis-cli KEYS "Customer:1:*"

print("\nKEYS Product:*:price")
!redis-cli KEYS "Product:*:price"

In [ ]:
print("🖥️ Database Info")
print("="*60)

print("\nDBSIZE - Total number of keys:")
!redis-cli DBSIZE

print("\nINFO keyspace:")
!redis-cli INFO keyspace

## 🖥️ Interactive Terminal Mode

To open an **interactive Redis terminal** in Colab:

1. Click the **Terminal** icon in the left sidebar (looks like a command prompt)
2. Or run the cell below and type commands one by one

In [ ]:
# Interactive command input - Run this cell, then type commands!
print("🖥️ INTERACTIVE REDIS CLI")
print("="*60)
print("Type Redis commands below (or 'quit' to exit)")
print("Examples: GET Customer:1:first_name")
print("          KEYS Product:*")
print("          DBSIZE")
print("="*60)

while True:
    cmd = input("\nredis> ")
    if cmd.lower() in ['quit', 'exit', 'q']:
        print("Bye!")
        break
    if cmd.strip():
        result = !redis-cli {cmd}
        for line in result:
            print(line)

---
# 🏗️ Part 8: Redis Cluster Architecture

## 8-Node Cluster (4 Masters + 4 Replicas)

```
┌─────────────┐  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐
│  MASTER 0   │  │  MASTER 1   │  │  MASTER 2   │  │  MASTER 3   │
│  Port 7000  │  │  Port 7001  │  │  Port 7002  │  │  Port 7003  │
│Slots 0-4095 │  │Slots 4096-  │  │Slots 8192-  │  │Slots 12288- │
│             │  │   8191      │  │  12287      │  │   16383     │
└──────┬──────┘  └──────┬──────┘  └──────┬──────┘  └──────┬──────┘
       │                │                │                │
       ▼                ▼                ▼                ▼
┌─────────────┐  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐
│  REPLICA 0  │  │  REPLICA 1  │  │  REPLICA 2  │  │  REPLICA 3  │
│  Port 7004  │  │  Port 7005  │  │  Port 7006  │  │  Port 7007  │
└─────────────┘  └─────────────┘  └─────────────┘  └─────────────┘
```

### Hash-Based Partitioning (Sharding):
- **Total slots:** 16384
- **Formula:** `slot = CRC16(key) mod 16384`
- Each key is assigned to a slot → slot belongs to a master node

### Replication:
- Each master has one replica for **high availability**
- If master fails → replica is promoted automatically

In [ ]:
# Simulate hash slot calculation
def crc16_simple(data):
    crc = 0
    for byte in data.encode():
        crc = ((crc << 5) + crc + byte) & 0xFFFF
    return crc

def get_slot(key):
    return crc16_simple(key) % 16384

def get_node(slot):
    if slot < 4096:
        return "Master 0 (Port 7000)"
    elif slot < 8192:
        return "Master 1 (Port 7001)"
    elif slot < 12288:
        return "Master 2 (Port 7002)"
    else:
        return "Master 3 (Port 7003)"

print("🎯 HASH SLOT DISTRIBUTION")
print("="*75)
print(f"{'Key':<35} {'Slot':>8}   {'Node'}")
print("-"*75)

keys_to_check = [
    "Customer:1:first_name",
    "Customer:2:email",
    "Product:101:product_name",
    "Product:103:price",
    "Order:1001:customer_id",
    "Order:1005:status"
]

for key in keys_to_check:
    slot = get_slot(key)
    node = get_node(slot)
    print(f"{key:<35} {slot:>8}   {node}")

---
# 📊 Part 9: Complete Mapping Summary

In [ ]:
print("="*80)
print("📊 COMPLETE KEY-VALUE MAPPING SUMMARY")
print("Format: TableName:TupleID:Attribute → Value")
print("="*80)

print("\n👥 CUSTOMER TABLE")
print("-"*80)
for cid in range(1, 6):
    print(f"\nCustomer {cid}:")
    for attr in ['first_name', 'last_name', 'email', 'phone', 'city', 'country']:
        key = f"Customer:{cid}:{attr}"
        val = r.get(key)
        print(f"   {key:<30} → \"{val}\"")

In [ ]:
print("📦 PRODUCT TABLE")
print("-"*80)
for pid in [101, 102, 103, 104, 105, 106]:
    print(f"\nProduct {pid}:")
    for attr in ['product_name', 'category', 'price', 'stock_quantity']:
        key = f"Product:{pid}:{attr}"
        val = r.get(key)
        print(f"   {key:<35} → \"{val}\"")

In [ ]:
print("🛒 ORDER TABLE")
print("-"*80)
for oid in [1001, 1002, 1003, 1004, 1005, 1006, 1007]:
    print(f"\nOrder {oid}:")
    for attr in ['customer_id', 'order_date', 'status', 'total_amount']:
        key = f"Order:{oid}:{attr}"
        val = r.get(key)
        print(f"   {key:<30} → \"{val}\"")

---
# ✅ Conclusion

## What We Demonstrated:

| # | Topic | Details |
|---|-------|--------|
| 1 | **Relational Model** | 3 tables: Customer (5), Product (6), Order (7) |
| 2 | **Mapping Strategy** | `TableName:TupleID:Attribute` → Value |
| 3 | **Migration** | Each column becomes a separate key-value pair |
| 4 | **GET by Key** | Direct lookup: `GET Customer:1:email` |
| 5 | **GET by Value** | Pattern matching: `KEYS Customer:*:email` |
| 6 | **Sharding** | Hash-based partitioning (16384 slots, 4 masters) |
| 7 | **Replication** | 4 replicas for high availability |

## Key Format:
```
TableName:TupleID:Attribute → Value

Examples:
Customer:1:first_name → "Ahmed"
Customer:1:email → "ahmed.rashid@email.com"
Product:101:price → "4500.0"
Order:1001:status → "delivered"
```

---
**Thank you!**